# Biosignal processing


#### Dependencies

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import systole
import plotly
import peakdet

In [4]:
%pylab inline
sns.set_style('darkgrid')
pylab.rcParams['figure.figsize'] = (12.0, 6.0)  
plt.rcParams.update({'font.size': 14})

Populating the interactive namespace from numpy and matplotlib


In [138]:
%run ~/cneuromod/ds_prep/physio/preproc/neuromod_process.py

In [6]:
filename = "/scratch/flesp/physio_data/friends/sub-05/ses-013/neuromod_video52021-02-15T16_01_28_10"

In [7]:
json=pd.read_json(f"{filename}.json")

In [8]:
fs = json['SamplingFrequency'][0]

In [9]:
bio_df = pd.read_csv(f"{filename}.tsv.gz",
                     sep='\t',
                     compression='gzip',
                     header=None,
                     names=json.Columns)

In [159]:
signals, info_corrected = neuromod_ppg_process(bio_df['PPG'], fs)

Cleaning the RR interval time series using 2 iterations.
 - Iteration 1 - 
... correcting 7 missed interval(s).
... correcting 1 extra interval(s).
... correcting 9 ectopic interval(s).
... correcting 4 short interval(s).
... correcting 30 long interval(s).
 - Iteration 2 - 
... correcting 1 missed interval(s).
... correcting 8 ectopic interval(s).
... correcting 2 long interval(s).


In [41]:
plot = systole.plots.plot_raw(signals['PPG_Clean'], backend='bokeh', show_heart_rate=True, show_artefacts=True, sfreq=fs)

In [43]:
from bokeh.io import output_notebook
from bokeh.plotting import show, output_file, save
output_notebook()

show(plot)

Loading BokehJS ...

In [177]:
plot_rr = systole.plots.plot_rr(info_corrected['clean_rr'],input_type='rr_ms',backend='bokeh', show_artefacts=True)

In [179]:
output_notebook()

show(plot_rr)

Loading BokehJS ...

In [ ]:
bio_df, bio_info = neuromod_bio_process(df=bio_df, sampling_rate=fs)

In [ ]:
bio_df.keys()